In [ ]:
# CNN Training Loop + Validation + Logging

In [1]:
import os, json, random
import numpy as np
import pandas as pd
import tensorflow as tf

SEED = 42
os.environ["PYTHONHASHSEED"] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
RUN_NAME = "cnn_run_01"
LOG_DIR = os.path.join("reports", "training_logs", RUN_NAME)
MODEL_DIR = "models"
os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

config = {
    "seed": SEED,
    "epochs": 15,
    "batch_size": 64,
    "learning_rate": 1e-3,
    "run_name": RUN_NAME,
}

with open(os.path.join(LOG_DIR, "config.json"), "w") as f:
    json.dump(config, f, indent=2)


In [3]:
# Placeholder example: (N, window_len, channels)
N = 5000
window_len = 256

X = np.random.randn(N, window_len, 1).astype(np.float32)
y = np.random.randint(0, 2, size=(N,)).astype(np.int32)

split = int(N * 0.8)
X_train, y_train = X[:split], y[:split]
X_val, y_val = X[split:], y[split:]


In [4]:
inputs = tf.keras.Input(shape=(X_train.shape[1], X_train.shape[2]))
x = tf.keras.layers.Conv1D(32, 5, activation="relu")(inputs)
x = tf.keras.layers.MaxPool1D(2)(x)
x = tf.keras.layers.Conv1D(64, 5, activation="relu")(x)
x = tf.keras.layers.GlobalAveragePooling1D()(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)

model = tf.keras.Model(inputs, outputs)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=config["learning_rate"]),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 256, 1)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 252, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 126, 32)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 122, 64)        │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 64)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,721 (57.50 KB)

 Trainable params: 14,721 (57.50 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
callbacks = [
    tf.keras.callbacks.CSVLogger(os.path.join(LOG_DIR, "metrics.csv")),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=os.path.join(MODEL_DIR, "best_model.keras"),
        monitor="val_loss",
        save_best_only=True
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=3,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=config["epochs"],
    batch_size=config["batch_size"],
    callbacks=callbacks,
    verbose=1
)


Epoch 1/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 4s 31ms/step - accuracy: 0.5130 - loss: 0.6924 - val_accuracy: 0.5100 - val_loss: 0.6929
Epoch 2/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.4880 - loss: 0.6945 - val_accuracy: 0.5100 - val_loss: 0.6929
Epoch 3/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.4899 - loss: 0.6938 - val_accuracy: 0.5100 - val_loss: 0.6931
Epoch 4/15
63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.4870 - loss: 0.6934 - val_accuracy: 0.5100 - val_loss: 0.6931


In [7]:
print("Saved files:")
print(" -", os.path.join(LOG_DIR, "metrics.csv"))
print(" -", os.path.join(LOG_DIR, "config.json"))
print(" -", os.path.join(MODEL_DIR, "best_model.keras"))


Saved files:
 - reports/training_logs/cnn_run_01/metrics.csv
 - reports/training_logs/cnn_run_01/config.json
 - models/best_model.keras
